### Data Mining and Machine Learning
### Ensembles of classifiers
#### Datasets:  Diabetes and Landsat
#### Edgar Acuna
#### April 2019

In [169]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
h2o.init(ip="localhost", port=54323)
#h2o.no_progress()

Checking whether there is an H2O instance running at http://localhost:54323. connected.


H2O cluster uptime:,1 hour 9 mins
H2O cluster timezone:,America/La_Paz
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.2
H2O cluster version age:,2 months and 15 days
H2O cluster name:,H2O_from_python_edgar2017_frvj4w
H2O cluster total nodes:,1
H2O cluster free memory:,1.637 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [170]:
x=[5,3,12,13,21,31,8,9,15,17,24,32] 

In [171]:
boot1=np.random.choice(x,12)
print(boot1)

[32 13 17  9 32 24  5  8 12  3 12  5]


In [172]:
np.unique(boot1)

array([ 3,  5,  8,  9, 12, 13, 17, 24, 32])

In [173]:
boot2=np.random.choice(x,12)
print(boot2)

[12 12 21 12  3 32 31  5 24 32  9  3]


In [174]:
np.unique(boot2)

array([ 3,  5,  9, 12, 21, 24, 31, 32])

Note: Approximately 37% of the  instances of the training sample  L DO NOT appear in any bootstrap sample. In the above examples 16.67% and 41.67% of instances do not appear in each  of the bootstrap samples.

### I. Bagging for Diabetes using trees and scikit learn

In [175]:
url= "http://academic.uprm.edu/eacuna/diabetes.dat"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_table(url, names=names,header=None)
#The response varaiable must be binary  (0,1)
y=data['class']-1
X=data.iloc[:,0:8]
modeltree = tree.DecisionTreeClassifier()
bagging = BaggingClassifier(modeltree,n_estimators=100)

In [177]:
# Accuracy rate by resubstitution
bagging.fit(X, y)
predictions = bagging.predict(X)
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       268

   micro avg       1.00      1.00      1.00       768
   macro avg       1.00      1.00      1.00       768
weighted avg       1.00      1.00      1.00       768



In [178]:
#Estimating the accuracy by cross validation
kfold = model_selection.KFold(n_splits=10, random_state=99)
results = model_selection.cross_val_score(bagging, X, y, cv=kfold)
print(results.mean())

0.7603896103896104


#### Out-of-Bag accuracy

In [179]:
bagging1 = BaggingClassifier(modeltree,n_estimators=50, oob_score=True)
bagging1.fit(X, y)
bagging1.oob_score_

0.7643229166666666

### II. AdaBoosting para Diabetes usando scikit-learn

In [180]:
adaboost = AdaBoostClassifier(modeltree,n_estimators=100,learning_rate=1)
adaboost.fit(X, y)
predictions = adaboost.predict(X)
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       268

   micro avg       1.00      1.00      1.00       768
   macro avg       1.00      1.00      1.00       768
weighted avg       1.00      1.00      1.00       768



In [181]:
#Estimating the accuracy by cross-validation
kfold = model_selection.KFold(n_splits=10, random_state=999)
results = model_selection.cross_val_score(adaboost, X, y, cv=kfold)
print(results.mean())

0.6938311688311688


### III. Gradient Boosting para Diabetes usando scikit-learn

In [182]:
gboost = GradientBoostingClassifier(n_estimators=100)
#X_train, X_train_lr, y_train, y_train_lr = train_test_split(X,y,test_size=0.5)
gboost.fit(X, y)
predictions = gboost.predict(X)
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93       500
           1       0.91      0.81      0.86       268

   micro avg       0.91      0.91      0.91       768
   macro avg       0.91      0.88      0.89       768
weighted avg       0.91      0.91      0.90       768



In [183]:
#Estimating the accueacy bt cross-validation
kfold = model_selection.KFold(n_splits=10, random_state=999)
results = model_selection.cross_val_score(gboost, X, y, cv=kfold)
print(results.mean())

0.7669002050580999


### IV  Gradient Boosting for diabetes using h2o

In [184]:
diabetes = h2o.import_file("https://academic.uprm.edu/eacuna/diabetes.dat")
myx=['C1','C2','C3','C4','C5','C6','C7','C8']
diabetes['C9']=diabetes['C9'].asfactor()
myy="C9"
gbm1 = H2OGradientBoostingEstimator(model_id="gbm_covType_v1",ntrees = 100, max_depth=4,nfolds=10, sample_rate = 1,col_sample_rate = 1,seed=20000)
gbm1.train(myx, myy, training_frame=diabetes)
y_pred=gbm1.predict(diabetes)
print((y_pred['predict']==diabetes['C9']).mean())

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
[0.9388020833333334]


In [185]:
#Accuracy ny resubstitution
gbm1.model_performance(diabetes)


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.06189488461506235
RMSE: 0.24878682564609877
LogLoss: 0.2278218904572922
Mean Per-Class Error: 0.06844776119402984
AUC: 0.9836082089552238
pr_auc: 0.9663482460483861
Gini: 0.9672164179104477
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4582730505624667: 


,1,2,Error,Rate
1,482.0,18.0,0.036,(18.0/500.0)
2,28.0,240.0,0.1045,(28.0/268.0)
Total,510.0,258.0,0.0599,(46.0/768.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4582731,0.9125475,171.0
max f2,0.3331835,0.9291396,206.0
max f0point5,0.5382824,0.9364407,148.0
max accuracy,0.4582731,0.9401042,171.0
max precision,0.9859219,1.0,0.0
max recall,0.1045607,1.0,309.0
max specificity,0.9859219,1.0,0.0
max absolute_mcc,0.4582731,0.8673905,171.0
max min_per_class_accuracy,0.4126285,0.9291045,185.0
max mean_per_class_accuracy,0.4126285,0.9315522,185.0


Gains/Lift Table: Avg response rate: 34.90 %, avg score: 34.90 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0104167,0.9743136,2.8656716,2.8656716,1.0,0.9792077,1.0,0.9792077,0.0298507,0.0298507,186.5671642,186.5671642
,2,0.0208333,0.9648206,2.8656716,2.8656716,1.0,0.9699281,1.0,0.9745679,0.0298507,0.0597015,186.5671642,186.5671642
,3,0.03125,0.9559245,2.8656716,2.8656716,1.0,0.9600379,1.0,0.9697246,0.0298507,0.0895522,186.5671642,186.5671642
,4,0.0403646,0.9527817,2.8656716,2.8656716,1.0,0.9540077,1.0,0.9661756,0.0261194,0.1156716,186.5671642,186.5671642
,5,0.0507812,0.9411744,2.8656716,2.8656716,1.0,0.9461405,1.0,0.9620659,0.0298507,0.1455224,186.5671642,186.5671642
,6,0.1002604,0.8979212,2.8656716,2.8656716,1.0,0.9198390,1.0,0.9412266,0.1417910,0.2873134,186.5671642,186.5671642
,7,0.1510417,0.8357249,2.8656716,2.8656716,1.0,0.8691100,1.0,0.9169805,0.1455224,0.4328358,186.5671642,186.5671642
,8,0.2005208,0.7529427,2.8656716,2.8656716,1.0,0.7897352,1.0,0.8855823,0.1417910,0.5746269,186.5671642,186.5671642
,9,0.3007812,0.5329792,2.5679395,2.7664276,0.8961039,0.6480360,0.9653680,0.8064002,0.2574627,0.8320896,156.7939523,176.6427602
,10,0.3997396,0.3458841,1.2065986,2.3802810,0.4210526,0.4361782,0.8306189,0.7147492,0.1194030,0.9514925,20.6598586,138.0281005


In [186]:
#Mostrando la matrix de confusion para estimar la precision out-of-bag y por validacion crizada
gbm1.confusion_matrix

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  gbm_covType_v1


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.06189488480582112
RMSE: 0.24878682602947674
LogLoss: 0.2278218932658851
Mean Per-Class Error: 0.06844776119402984
AUC: 0.9836082089552238
pr_auc: 0.9663482460483861
Gini: 0.9672164179104477
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4582730550290878: 


,1,2,Error,Rate
1,482.0,18.0,0.036,(18.0/500.0)
2,28.0,240.0,0.1045,(28.0/268.0)
Total,510.0,258.0,0.0599,(46.0/768.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4582731,0.9125475,171.0
max f2,0.3331834,0.9291396,206.0
max f0point5,0.5382824,0.9364407,148.0
max accuracy,0.4582731,0.9401042,171.0
max precision,0.9859219,1.0,0.0
max recall,0.1045607,1.0,309.0
max specificity,0.9859219,1.0,0.0
max absolute_mcc,0.4582731,0.8673905,171.0
max min_per_class_accuracy,0.4126285,0.9291045,185.0
max mean_per_class_accuracy,0.4126285,0.9315522,185.0


Gains/Lift Table: Avg response rate: 34.90 %, avg score: 34.90 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
,2019-04-03 19:02:28,2.357 sec,0.0,0.4766408,0.6467994,0.5,0.0,1.0,0.6510417
,2019-04-03 19:02:28,2.360 sec,1.0,0.4583666,0.6094445,0.8705560,0.5840419,2.6268657,0.21875
,2019-04-03 19:02:28,2.363 sec,2.0,0.4430368,0.5792476,0.8758731,0.6118444,2.7034638,0.2135417
,2019-04-03 19:02:28,2.365 sec,3.0,0.4300693,0.5542099,0.8826418,0.6160802,2.7034638,0.2083333
,2019-04-03 19:02:28,2.367 sec,4.0,0.4193521,0.5337535,0.8839366,0.6425409,2.8005427,0.2057292
---,---,---,---,---,---,---,---,---,---
,2019-04-03 19:02:28,2.689 sec,96.0,0.2537534,0.2344594,0.9816940,0.9630855,2.8656716,0.0651042
,2019-04-03 19:02:28,2.693 sec,97.0,0.2533385,0.2337444,0.9817873,0.9632840,2.8656716,0.0651042
,2019-04-03 19:02:28,2.697 sec,98.0,0.2523625,0.2323599,0.9823284,0.9641129,2.8656716,0.0638021
,2019-04-03 19:02:28,2.700 sec,99.0,0.2508351,0.2304203,0.9828806,0.9650847,2.8656716,0.0625



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
C2,253.0619049,1.0,0.4011016
C6,133.0974426,0.5259482,0.2109587
C8,78.7900696,0.3113470,0.1248818
C7,65.5236588,0.2589234,0.1038546
C1,35.4766197,0.1401895,0.0562302
C5,30.2431660,0.1195090,0.0479352
C3,22.3595104,0.0883559,0.0354397
C4,12.3648167,0.0488608,0.0195982


<bound method H2OBinomialModel.confusion_matrix of >

### V. Bagging  using Decision Trees for Landsat (scikit-learn)

In [190]:
url='http://academic.uprm.edu/eacuna/landsat.txt'
data = pd.read_table(url, header=None,delim_whitespace=True)
y=data.iloc[:,36]-1
names=['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13',
            'C14','C15','C16','C17','C18','C19','C20','C21','C22','C23','C24','C25','C26','C27',
           'C28','C29', 'C30','C31','C32','C33','C34','C35','C36','C37']
X=data.iloc[:,0:36]
modeltree = tree.DecisionTreeClassifier()
bagging = BaggingClassifier(modeltree,n_estimators=100, max_features=1.0)
# Tasa de precision
bagging.fit(X, y)
predictions = bagging.predict(X)
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1072
           1       1.00      1.00      1.00       479
           2       1.00      1.00      1.00       961
           3       1.00      1.00      1.00       415
           4       1.00      1.00      1.00       470
           5       1.00      1.00      1.00      1038

   micro avg       1.00      1.00      1.00      4435
   macro avg       1.00      1.00      1.00      4435
weighted avg       1.00      1.00      1.00      4435



In [191]:
#Accuracy by resubstitution
kfold = model_selection.KFold(n_splits=10, random_state=99)
results = model_selection.cross_val_score(modeltree, X, y, cv=kfold)
print(results.mean())

0.7979826327456124


In [192]:
#accuracy by out-of-bag
bagging1 = BaggingClassifier(modeltree,n_estimators=50, oob_score=True)
bagging1.fit(X, y)
bagging1.oob_score_

0.8980834272829763

### VI. AdaBoosting for Landsat

In [193]:
adaboost = AdaBoostClassifier(modeltree,n_estimators=100,learning_rate=1)
adaboost.fit(X, y)
predictions = adaboost.predict(X)
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1072
           1       1.00      1.00      1.00       479
           2       1.00      1.00      1.00       961
           3       1.00      1.00      1.00       415
           4       1.00      1.00      1.00       470
           5       1.00      1.00      1.00      1038

   micro avg       1.00      1.00      1.00      4435
   macro avg       1.00      1.00      1.00      4435
weighted avg       1.00      1.00      1.00      4435



In [194]:
#accuracy by cross-validation
kfold = model_selection.KFold(n_splits=10, random_state=999)
results = model_selection.cross_val_score(adaboost, X, y, cv=kfold)
print(results.mean())

0.8045258576861285


In [195]:
gboost = GradientBoostingClassifier(n_estimators=100)
#X_train, X_train_lr, y_train, y_train_lr = train_test_split(X,y,test_size=0.5)
gboost.fit(X, y)
predictions = gboost.predict(X)
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1072
           1       1.00      1.00      1.00       479
           2       0.96      0.99      0.98       961
           3       0.97      0.87      0.91       415
           4       0.99      0.99      0.99       470
           5       0.97      0.98      0.98      1038

   micro avg       0.98      0.98      0.98      4435
   macro avg       0.98      0.97      0.98      4435
weighted avg       0.98      0.98      0.98      4435



In [196]:
#Estimating the accueacy bt cross-validation
kfold = model_selection.KFold(n_splits=10, random_state=999)
results = model_selection.cross_val_score(gboost, X, y, cv=kfold)
print(results.mean())

0.8645084700953776


### VII. Gradient Boostimg for Landsat using H2o

In [200]:
#Leyendo los datos
datos= h2o.import_file("http://academic.uprm.edu/eacuna/landsat.txt")
myx=['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13',
            'C14','C15','C16','C17','C18','C19','C20','C21','C22','C23','C24','C25','C26','C27',
           'C28','C29', 'C30','C31','C32','C33','C34','C35','C36']
datos['C37']=datos['C37'].asfactor()
myy="C37"
gbm2 = H2OGradientBoostingEstimator(model_id="gbm_covType_v1",ntrees = 100, max_depth=4,nfolds=10, sample_rate = 1,col_sample_rate = 1,seed=20000)
gbm2.train(myx, myy, training_frame=datos)
y_pred=gbm2.predict(datos)
print((y_pred['predict']==datos['C37']).mean())

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
[0.9950394588500564]


In [201]:
#Mostrando la matrix de confusion para estimar la precision out-of-bag y por validacion crizada
gbm2.confusion_matrix

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  gbm_covType_v1


ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.00798614637609013
RMSE: 0.08936524143138724
LogLoss: 0.044830497516567674
Mean Per-Class Error: 0.007389093948046522
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



1,2,3,4,5,6,Error,Rate
1072.0,0.0,0.0,0.0,0.0,0.0,0.0,"0 / 1,072"
0.0,479.0,0.0,0.0,0.0,0.0,0.0,0 / 479
0.0,0.0,961.0,0.0,0.0,0.0,0.0,0 / 961
0.0,0.0,7.0,399.0,0.0,9.0,0.0385542,16 / 415
0.0,0.0,0.0,0.0,470.0,0.0,0.0,0 / 470
0.0,0.0,4.0,2.0,0.0,1032.0,0.0057803,"6 / 1,038"
1072.0,479.0,972.0,401.0,470.0,1041.0,0.0049605,"22 / 4,435"


Top-6 Hit Ratios: 


k,hit_ratio
1,0.9950395
2,0.9995490
3,1.0
4,1.0
5,1.0
6,1.0



ModelMetricsMultinomial: gbm
** Reported on cross-validation data. **

MSE: 0.0720374623883169
RMSE: 0.26839795526105803
LogLoss: 0.242699011096459
Mean Per-Class Error: 0.11790647672150929
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



...

,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
,2019-04-03 19:11:40,26.873 sec,0.0,0.8333333,1.7917595,0.8063134
,2019-04-03 19:11:40,26.899 sec,1.0,0.7615863,1.4415167,0.1278467
,2019-04-03 19:11:40,26.923 sec,2.0,0.6986460,1.2151781,0.1149944
,2019-04-03 19:11:40,26.946 sec,3.0,0.6423963,1.0488657,0.1113867
,2019-04-03 19:11:40,26.968 sec,4.0,0.5920875,0.9191481,0.1059752
---,---,---,---,---,---,---
,2019-04-03 19:11:42,29.138 sec,96.0,0.0939514,0.0479037,0.0058625
,2019-04-03 19:11:42,29.161 sec,97.0,0.0926412,0.0470240,0.0056370
,2019-04-03 19:11:42,29.188 sec,98.0,0.0914996,0.0462301,0.0054115
,2019-04-03 19:11:42,29.211 sec,99.0,0.0901773,0.0453948,0.0049605



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
C17,2636.6804199,1.0,0.1909320
C22,1897.1207275,0.7195111,0.1373777
C20,1485.0837402,0.5632399,0.1075405
C18,1168.8540039,0.4433051,0.0846411
C34,1040.6865234,0.3946957,0.0753600
---,---,---,---
C8,63.4294167,0.0240565,0.0045932
C1,54.3748970,0.0206225,0.0039375
C11,53.4609451,0.0202759,0.0038713
C31,47.8458672,0.0181463,0.0034647



See the whole table with table.as_data_frame()


<bound method H2OMultinomialModel.confusion_matrix of >